### Run the following chunk of code to import any libraries and packages required to run the rest of the script.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Run the following chunk of code to generate the architecture used for the phase two classification network.

Here we are using a VGG16 architecture with pre-trained weights on the imagenet dataset. This transfer learning approach greatly reduces the required training data input on our end.

In [2]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(40,40,3))
base_model.trainable = False ## Not trainable weights

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(16, activation='relu')
dense_layer_2 = layers.Dense(8, activation='relu')
dense_layer_3 = layers.Dense(4, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer
])

### Run the following code to create the data generators, which will read in the training and testing data in batches.

The user will need to change the directory tag in the traindata and testdata variables to match the location of their. Please the possible confusion of names here, when we use test in the code, we really mean validate.

In [3]:
trdata = ImageDataGenerator(    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=True, vertical_flip=True, rescale=None,
    preprocessing_function=preprocess_input, data_format=None, validation_split=0.0, dtype=None)
traindata = trdata.flow_from_directory(directory="",target_size=(40,40))

tsdata = ImageDataGenerator(    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=True, vertical_flip=True, rescale=None,
    preprocessing_function=preprocess_input, data_format=None, validation_split=0.0, dtype=None)
testdata = tsdata.flow_from_directory(directory="",target_size=(40,40))

Found 6180 images belonging to 2 classes.
Found 1765 images belonging to 2 classes.


### Run the following code to train the pre-built VGG16 architecture and weights to classify cells in our problem.

The user may need to change the class_weight variable, to account for any particular class imbalances in your dataset.

In [4]:
class_weight = {0:1/1028, 1: 1/6917}

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True)

hist = model.fit(traindata, validation_data= testdata,epochs=1000,callbacks=[checkpoint,early],class_weight = class_weight)

Epoch 1/1000
194/194 [==============================] - ETA: 0s - loss: 6.8983e-05 - accuracy: 0.8764
Epoch 1: val_loss improved from inf to 0.16065, saving model to vgg16_1.h5
194/194 [==============================] - 27s 133ms/step - loss: 6.8983e-05 - accuracy: 0.8764 - val_loss: 0.1606 - val_accuracy: 0.9394
Epoch 2/1000
194/194 [==============================] - ETA: 0s - loss: 3.2391e-05 - accuracy: 0.9261
Epoch 2: val_loss improved from 0.16065 to 0.13912, saving model to vgg16_1.h5
194/194 [==============================] - 26s 133ms/step - loss: 3.2391e-05 - accuracy: 0.9261 - val_loss: 0.1391 - val_accuracy: 0.9496
Epoch 3/1000
194/194 [==============================] - ETA: 0s - loss: 2.3555e-05 - accuracy: 0.9505
Epoch 3: val_loss did not improve from 0.13912
194/194 [==============================] - 25s 129ms/step - loss: 2.3555e-05 - accuracy: 0.9505 - val_loss: 0.1400 - val_accuracy: 0.9235
Epoch 4/1000
194/194 [==============================] - ETA: 0s - loss: 2.0257e

### Run the following chunk of code to save the trained network architecture and weights.

The user will need to input their own target directory to save the network to.

In [5]:
model.save("")